# Transfer Learning

## Load Libraries



In [1]:
import importlib
import sys
import os
import numpy as np
import time

import torch
from torch import nn
import torch.optim as optim

from torch.utils import data
from torchvision import transforms

sys.path.append(os.path.join(os.getcwd(), ".."))

from distiller import apputils
import ai8x

kws20 = importlib.import_module("datasets.kws20-horsecough")

## Calculate weights

In [2]:
from pathlib import Path

raw_data_path = Path("../data/KWS_EQUINE/raw/")
class_file_count = {}

class_dirs = [d for d in raw_data_path.iterdir() if d.is_dir() and d.stem != "_background_noise_"]

for d in class_dirs:
    class_file_count[d] = len(list(d.iterdir()))

min_file_count = float(min(class_file_count.values()))

for d in class_dirs:
    class_file_count[d] = min_file_count / class_file_count[d]
    print(f"{d.stem}: {round(class_file_count[d], 7)}")

horse_cough: 0.3773585
horse_neigh: 0.7407407
human_cough: 1.0


## Generate processed dataset

In [3]:
train_batch_size = 32
train_loader, val_loader, test_loader, _ = apputils.get_data_loaders(
    kws20.KWS_HORSE_TF_get_datasets, ("../data", False), train_batch_size, 1, validation_split=0)
print(f"Dataset sizes:\n\ttraining={len(train_loader.sampler)}\n\tvalidation={len(val_loader.sampler)}\n\ttest={len(test_loader.sampler)}")

No key `noise_var` in input augmentation dictionary!  Using defaults: [Min: 0., Max: 1.]
No key `shift` in input augmentation dictionary! Using defaults: [Min:-0.1, Max: 0.1]
No key `strech` in input augmentation dictionary! Using defaults: [Min: 0.8, Max: 1.3]
Generating dataset from raw data samples for the first time. 
This process will take significant time (~60 minutes)...
data_len: 16384
------------- Label Size ---------------
horse_cough:  	53
horse_neigh:  	27
human_cough:  	20
------------------------------------------
Processing the label: horse_cough. 1 of 3
	1 of 53
Finished in 1.604 seconds.
(159, 128, 128)
Data concatenation finished in 0.001 seconds.
Processing the label: horse_neigh. 2 of 3
	1 of 27
Finished in 0.802 seconds.
(81, 128, 128)
Data concatenation finished in 0.002 seconds.
Processing the label: human_cough. 3 of 3
	1 of 20
Finished in 0.609 seconds.
(60, 128, 128)
Data concatenation finished in 0.008 seconds.
Dataset created.
Training+Validation: 19,  Test

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


In [5]:
classes = ["horse_cough", "horse_neigh", "human_cough"]

In [6]:
def count_params(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    return params

In [7]:
ai8x.set_device(device=85, simulate=False, round_avg=False)

mod = importlib.import_module("models.ai85net-kws20-kabayo")

model = mod.AI85KWS20Netv3(num_classes=37, num_channels=128, dimensions=(128, 1), bias=False)
print(f'Number of Model Params: {count_params(model)}')

# model, compression_scheduler, optimizer, start_epoch = apputils.load_checkpoint(
#             model, "logs/2023.01.10-013008/qat_best.pth.tar", model_device='cuda')

model, compression_scheduler, optimizer, start_epoch = apputils.load_checkpoint(
            model, "../logs/2023.01.23-053753/qat_best.pth.tar")

Configuring device: MAX78000, simulate=False.
Number of Model Params: 173568


## Replace FC layer and freeze the rest of the layers

In [8]:
def freeze_layer(layer):
    for p in layer.parameters():
        p.requires_grad = False

In [9]:
freeze_layer(model.voice_conv1)
freeze_layer(model.voice_conv2)
freeze_layer(model.voice_conv3)
freeze_layer(model.voice_conv4)
freeze_layer(model.kws_conv1)
# freeze_layer(model.kws_conv2)
# freeze_layer(model.kws_conv3)
# freeze_layer(model.kws_conv4)
model.fc = ai8x.Linear(256, len(classes), bias=False, wide=True)

model = model.to(device)

## Train the model

In [14]:
num_epochs = 50
epoch = 0
optimizer = optim.Adam(model.parameters(), lr=0.0001)
ms_lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20, 80], gamma=0.5)
criterion = torch.nn.CrossEntropyLoss(
    torch.Tensor((1, 1, 1))
)
criterion.to(device)

# criterion = torch.nn.CrossEntropyLoss(
#     torch.Tensor((1, 1, 0.01, 1, 1, 0.01, 0.01, 0.01))
# )
# criterion.to(device)

qat_policy = {
    'start_epoch': 20,
    'weight_bits': 8
    }

In [15]:
from sklearn.metrics import confusion_matrix

def plot_confusion(y_true, y_pred, classes):
    cf_matrix = confusion_matrix(y_true, y_pred, list(range(len(classes))))
    print(cf_matrix)

In [16]:
# for i, (input, target) in enumerate(train_loader):
#     print(np.shape(input), np.shape(target), sep=" ")

In [17]:
best_acc = 0
best_qat_acc = 0
for epoch in range(0, num_epochs):
    if epoch > 0 and epoch == qat_policy['start_epoch']:
        print('QAT is starting!')
        # Fuse the BN parameters into conv layers before Quantization Aware Training (QAT)
        ai8x.fuse_bn_layers(model)

        # Switch model from unquantized to quantized for QAT
        ai8x.initiate_qat(model, qat_policy)

        # Model is re-transferred to GPU in case parameters were added
        model.to(device)
    running_loss = []
    train_start = time.time()
    model.train()
    for idx, (inputs, target) in enumerate(train_loader):
        inputs = inputs.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        
        model_out = model(inputs)
        
        loss = criterion(model_out, target)
        loss.backward()
        optimizer.step()
        
        running_loss.append(loss.cpu().detach().numpy())

    mean_loss = np.mean(running_loss)
    train_end = time.time()
    print("Epoch: {}/{}\t LR: {}\t Train Loss: {:.4f}\t Dur: {:.2f} sec.".format(
        epoch+1, num_epochs, ms_lr_scheduler.get_lr(), mean_loss, (train_end-train_start)))
    
    model.eval()
    acc = 0.
    acc_weight = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, target in test_loader:
            inputs = inputs.to(device)
            target = target.to(device)
            model_out = model(inputs)
            target_out = torch.argmax(model_out, dim=1)
            
            y_pred.extend(target_out.cpu().numpy())
            y_true.extend(target.cpu().numpy())
            
            tp = torch.sum(target_out == target)
            acc_batch = (tp / target_out.numel()).detach().item()
            acc += target_out.shape[0] * acc_batch
            acc_weight += target_out.shape[0]
            
        total_acc = 100 * (acc / acc_weight)
        if epoch == qat_policy['start_epoch']: best_acc = 0
        if total_acc > best_acc:
            best_acc = total_acc
            checkpoint_extras = {'current_top1': best_acc,
                                 'best_top1': best_acc,
                                 'best_epoch': epoch}
            model_name = 'ai85net_kws_equine'
            model_prefix = f'{model_name}' if epoch < qat_policy['start_epoch'] else (f'qat_{model_name}')
            apputils.save_checkpoint(epoch, model_name, model, optimizer=optimizer,
                                     scheduler=None, extras=checkpoint_extras,
                                     is_best=True, name=model_prefix,
                                     dir='.')
            print(f'Best model saved with accuracy: {best_acc:.2f}%')
            
        print('\t\t Test Acc: {:.2f}'.format(total_acc))
        print("\t\tConfusion:")
        plot_confusion(y_true, y_pred, classes)
    ms_lr_scheduler.step()

Epoch: 1/50	 LR: [0.0001]	 Train Loss: 0.9043	 Dur: 10.08 sec.
Best model saved with accuracy: 66.67%
		 Test Acc: 66.67
		Confusion:
[[12  0  0]
 [ 2  0  1]
 [ 3  0  0]]
Epoch: 2/50	 LR: [0.0001]	 Train Loss: 0.9107	 Dur: 9.97 sec.
		 Test Acc: 66.67
		Confusion:
[[12  0  0]
 [ 2  0  1]
 [ 3  0  0]]
Epoch: 3/50	 LR: [0.0001]	 Train Loss: 0.8813	 Dur: 9.97 sec.
		 Test Acc: 66.67
		Confusion:
[[12  0  0]
 [ 2  0  1]
 [ 3  0  0]]
Epoch: 4/50	 LR: [0.0001]	 Train Loss: 0.8859	 Dur: 10.14 sec.
		 Test Acc: 66.67
		Confusion:
[[12  0  0]
 [ 2  0  1]
 [ 3  0  0]]
Epoch: 5/50	 LR: [0.0001]	 Train Loss: 0.8805	 Dur: 9.87 sec.
		 Test Acc: 55.56
		Confusion:
[[10  2  0]
 [ 2  0  1]
 [ 3  0  0]]
Epoch: 6/50	 LR: [0.0001]	 Train Loss: 0.8541	 Dur: 9.82 sec.
		 Test Acc: 55.56
		Confusion:
[[10  2  0]
 [ 2  0  1]
 [ 3  0  0]]
Epoch: 7/50	 LR: [0.0001]	 Train Loss: 0.8348	 Dur: 9.79 sec.
		 Test Acc: 55.56
		Confusion:
[[10  2  0]
 [ 2  0  1]
 [ 3  0  0]]
Epoch: 8/50	 LR: [0.0001]	 Train Loss: 0.8